# Geocoding and Isolines Services

This example illustrates how to combine both Geocoding and Isolines Data Services

> Note: CARTO Account credentials are needed to reproduce this example. https://carto.com/signup

In [1]:
import pandas
from cartoframes.auth import set_default_credentials

# set_default_credentials('johnsmith', '1a2b3c4d5e6f7g8h9i0j')
set_default_credentials(
    username='elena-carto',
    api_key='97293f98929325451d1db4fd42b66cd1168e13d6'
)

In [2]:
from cartoframes.data.services import Geocoding

df = pandas.DataFrame([
         ['Calle Serrano 15'],
         ['Calle de San Pedro 21'],
         ['Calle Gran Vía 46'],
         ['Paseo de la Castellana 200'],
         ['Calle Ntra. Sra. del Carmen 7'],
         ['Calle de San Germán 12'],
         ['Calle de Bravo Murillo 377'],
    ],
    columns=['address']
)

gc = Geocoding()
df_data, df_metadata = gc.geocode(df, street='address', city={'value': 'Madrid'}, country={'value': 'Spain'})

ImportError: cannot import name 'Dataset' from 'cartoframes.data' (/Users/elena/Projects/CARTO/carto-data-science-stack/cartoframes/cartoframes/data/__init__.py)

In [ ]:
print(df_metadata)

In [ ]:
df_data.head()

In [ ]:
df_data.viz()

In [ ]:
from cartoframes.data.services import Isolines

iso_service = Isolines()
isochrones = iso_service.isochrones(df_data, [100, 200, 300], mode='walk', dry_run=True)

print('Available Quota: {0}'.format(iso_service.available_quota()))
print('Required Quota: {0}'.format(isochrones.metadata.get('required_quota')))

In [ ]:
isochrones = iso_service.isochrones(df_data, [100, 200, 300], mode='walk')

print(isochrones.data)

In [ ]:
Map([
    Layer(isochrones.data),
    Layer(ds)
])

In [ ]:
Map([
    Layer(isochrones.data, 'color: opacity(green, 0.2) strokeWidth: 1'),
    Layer(ds, 'width: 1 strokeWidth: 3')
])